# Eliminação Gaussiana

Método de escalonamento utilizado na solução de sistemas lineares no formato $Ax = b$ com $ A \in \mathbb{R}, b \in \mathbb{R}^m, x \in \mathbb{R}^n$.

## BLAS ( *Basic Linear Algebra Subroutines*)

### BLAS nível 1 - $O(n)$

**Exemplos:**

$$ z = \alpha x + y \Rightarrow z_i = \alpha x_i + y_i $$

$$ \beta = x^Ty \Rightarrow \sum_{i=1}^{n}x_iy_i $$

### BLAS nível 2 - $O(n^2)$

**Exemplos:**

$$ y = Ax \Rightarrow y_i = \sum_{j=1}^{n}a_{ij}x_j = (a_{i.})^Tx_j $$

### BLAS nível 3 - $O(n^3)$

**Exemplos:**

$$ C = AB \Rightarrow c_{ij} = \sum_{k=1}^{n}a_{ik}b_{kj} $$
$$ A \in \mathbb{R}^{m \times n}, B \in \mathbb{R}^{n \times p}, C \in \mathbb{R}^{m \times p} $$

In [2]:
A = rand(3, 5)
B = rand(5, 4)
 A * B

3×4 Array{Float64,2}:
 0.995753  1.59089  0.782919  2.03146
 0.710461  1.44819  0.870257  2.05414
 0.953441  1.9885   0.972007  2.40502

In [3]:
# Exemplo de operação de BLAS nível 3
# C = AB

# iniciamos C com zeros
C = zeros(3, 4)

# loop para executar a multiplicação dos elementos de A e B
for i = 1:3, j = 1:4
    C[i,j] = sum(A[i,k] * B[k,j] for k = 1:5)
end
C

3×4 Array{Float64,2}:
 0.995753  1.59089  0.782919  2.03146
 0.710461  1.44819  0.870257  2.05414
 0.953441  1.9885   0.972007  2.40502

In [4]:
Float64.size

8

In [5]:
n = 100000
Float64.size * (3n - 2) / 1024 / 1024, Float64.size * n^2 / 1024 / 1024 /1024

(2.2888031005859375, 74.50580596923828)

Como podemos ver, para matrizes muito grandes precisamos nos preocupar com o espaço de armazenamento na memória. No exemplo acima, para uma matriz de 100.000 elementos, precisaríamos de somente 2 Mb para armazenar as três diagonais, porém, necessitamos de mais de 70 Gb para armazenar a matriz na sua completude.

In [6]:
import Pkg;
Pkg.add("BenchmarkTools")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100,0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [7]:
using BenchmarkTools

In [8]:
# Funções para calcular y = Ax, fazendo os loops em ordem diferente.
# A ideia é fazer o 'for' de j dentro do 'for' de i, e vice-versa.

# calcula y = Ax fazendo a iteração de primeiro i, depois j
function multAx1(y, A, x)
    m, n = size(A)
    for i = 1:m
        y[i] = 0.0
        for j = 1:n
            y[i] += A[i,j] * x[j]
        end
    end
end

# calcula y = Ax fazendo a iteração de primeiro j, depois i
function multAx2(y, A, x)
    m, n = size(A)
    
    for i = 1:m
        y[i] = 0.0
    end
    
    for j = 1:n
        for i = 1:m
            y[i] += A[i,j] * x[j]
        end
    end
end

multAx2 (generic function with 1 method)

In [9]:
n = 1000
A = rand(n, n)
x = rand(n)

1000-element Array{Float64,1}:
 0.3345830869337887
 0.7567331850149634
 0.14960926948328224
 0.7861313296303412
 0.06213140864185651
 0.5168874229552267
 0.9385421174041102
 0.2745180789221662
 0.9401558077229677
 0.5339649214974316
 0.33074219221550827
 0.21852227632453158
 0.5827517472563815
 ⋮
 0.587890611160244
 0.48290565513465467
 0.005349230369154023
 0.6689554331511762
 0.6626479862026922
 0.4860010395826708
 0.18168677448170523
 0.3417785778679421
 0.18189838728141772
 0.3677184168633696
 0.7140552206934214
 0.3693840522810534

In [10]:
y1 = zeros(n)
y2 = zeros(n)
multAx1(y1, A, x)
multAx2(y2, A, x)
y1 ≈ y2 ≈ A * x

true

In [11]:
y = zeros(n)
@benchmark multAx1(y, $A, $x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.417 ms (0.00% GC)
  median time:      6.030 ms (0.00% GC)
  mean time:        7.087 ms (0.00% GC)
  maximum time:     26.370 ms (0.00% GC)
  --------------
  samples:          706
  evals/sample:     1

In [12]:
y = zeros(n)
@benchmark multAx2(y, $A, $x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.226 ms (0.00% GC)
  median time:      2.003 ms (0.00% GC)
  mean time:        2.564 ms (0.00% GC)
  maximum time:     8.742 ms (0.00% GC)
  --------------
  samples:          1947
  evals/sample:     1

Realizar o loop de j mais externamente e o de i mais internamente auxilia o computador a otimizar os cálculos feitos. Como vemos nos testes de benchmark acima, a função multAx2 foi mais rápida que a função multAx1, somente pela forma como distribuímos a operação de multiplicação. Portanto, é importante atentar-se à forma de escrever as operações.

In [13]:
using SparseArrays

In [14]:
A = sprand(10, 10, 0.3)

10×10 SparseMatrixCSC{Float64,Int64} with 38 stored entries:
  [2 ,  1]  =  0.536976
  [4 ,  1]  =  0.988429
  [5 ,  1]  =  0.756865
  [8 ,  1]  =  0.439143
  [10,  1]  =  0.0213314
  [1 ,  2]  =  0.453316
  [4 ,  2]  =  0.274456
  [7 ,  2]  =  0.40178
  [1 ,  3]  =  0.442188
  [3 ,  3]  =  0.65759
  [7 ,  3]  =  0.28765
  [8 ,  3]  =  0.518579
  ⋮
  [4 ,  6]  =  0.166482
  [5 ,  6]  =  0.600658
  [8 ,  6]  =  0.89537
  [2 ,  8]  =  0.429552
  [4 ,  8]  =  0.00431129
  [5 ,  8]  =  0.96587
  [7 ,  8]  =  0.42297
  [5 ,  9]  =  0.670559
  [9 ,  9]  =  0.383742
  [3 , 10]  =  0.619325
  [4 , 10]  =  0.776877
  [5 , 10]  =  0.676678
  [10, 10]  =  0.493671

In [15]:
rows, cols, vals = findnz(A)

([2, 4, 5, 8, 10, 1, 4, 7, 1, 3  …  2, 4, 5, 7, 5, 9, 3, 4, 5, 10], [1, 1, 1, 1, 1, 2, 2, 2, 3, 3  …  8, 8, 8, 8, 9, 9, 10, 10, 10, 10], [0.5369757323509659, 0.9884286984806283, 0.7568650254851217, 0.43914340999589063, 0.021331439787203976, 0.4533158468793741, 0.2744562916500217, 0.40178026641005404, 0.44218750961753006, 0.6575895243264445  …  0.42955177782766607, 0.00431128687706317, 0.9658700136759955, 0.42296974872603954, 0.6705592067705797, 0.38374155620324113, 0.6193251284718935, 0.7768767671102965, 0.6766781546367748, 0.49367059561599236])

**Exercício:** Fazer as 6 ordens da $C = AB$.

In [16]:
n = 100
z = 50
m = 200

200

In [17]:
A = rand(n, z)
B = rand(z, m);

In [18]:
# Funções para calcular C = AB, fazendo os loops em ordem diferente.

# calcula C = AB fazendo a iteração i -> j -> k
function multAB1(C, A, B)
    # loop para executar a multiplicação dos elementos de A e B
    for i = 1:n
        for j = 1:m
            for k = 1:z
                C[i,j] += A[i,k] * B[k,j]
            end
        end
    end
end

# calcula C = AB fazendo a iteração i -> k -> j
function multAB2(C, A, B)
    # loop para executar a multiplicação dos elementos de A e B
    for i = 1:n
        for k = 1:z
            for j = 1:m
                C[i,j] += A[i,k] * B[k,j]
            end
        end
    end
end

# calcula C = AB fazendo a iteração k -> i -> j
function multAB3(C, A, B)
    # loop para executar a multiplicação dos elementos de A e B
    for k = 1:z
        for i = 1:n
            for j = 1:m
                C[i,j] += A[i,k] * B[k,j]
            end
        end
    end
end

# calcula C = AB fazendo a iteração k -> j -> i
function multAB4(C, A, B)
    # loop para executar a multiplicação dos elementos de A e B
    for k = 1:z
        for j = 1:m
            for i = 1:n
                C[i,j] += A[i,k] * B[k,j]
            end
        end
    end
end

# calcula C = AB fazendo a iteração j -> i -> k
function multAB5(C, A, B)
    # loop para executar a multiplicação dos elementos de A e B
    for j = 1:m
        for i = 1:n
            for k = 1:z
                C[i,j] += A[i,k] * B[k,j]
            end
        end
    end
end

# calcula C = AB fazendo a iteração j -> k -> i
function multAB6(C, A, B)
    # loop para executar a multiplicação dos elementos de A e B
    for j = 1:m
        for k = 1:z
            for i = 1:n
                C[i,j] += A[i,k] * B[k,j]
            end
        end
    end
end

multAB6 (generic function with 1 method)

In [19]:
C1 = zeros(n, m)
C2 = zeros(n, m)
C3 = zeros(n, m)
C4 = zeros(n, m)
C5 = zeros(n, m)
C6 = zeros(n, m)

multAB1(C1, A, B)
multAB2(C2, A, B)
multAB3(C3, A, B)
multAB4(C4, A, B)
multAB5(C5, A, B)
multAB6(C6, A, B)

C1 ≈ C2 ≈ C3 ≈ C4 ≈ C5 ≈ C6 ≈ A * B

true

In [20]:
C = zeros(n, m);

In [21]:
@benchmark multAB1(C, $A, $B)

BenchmarkTools.Trial: 
  memory estimate:  108.04 MiB
  allocs estimate:  6040201
  --------------
  minimum time:     294.483 ms (7.75% GC)
  median time:      308.019 ms (7.41% GC)
  mean time:        310.827 ms (7.20% GC)
  maximum time:     330.473 ms (4.74% GC)
  --------------
  samples:          17
  evals/sample:     1

In [22]:
@benchmark multAB2(C, $A, $B)

BenchmarkTools.Trial: 
  memory estimate:  107.12 MiB
  allocs estimate:  6010201
  --------------
  minimum time:     312.952 ms (5.71% GC)
  median time:      341.562 ms (5.80% GC)
  mean time:        339.976 ms (6.04% GC)
  maximum time:     366.566 ms (8.05% GC)
  --------------
  samples:          15
  evals/sample:     1

In [23]:
@benchmark multAB3(C, $A, $B)

BenchmarkTools.Trial: 
  memory estimate:  107.12 MiB
  allocs estimate:  6010101
  --------------
  minimum time:     294.372 ms (6.57% GC)
  median time:      332.594 ms (6.04% GC)
  mean time:        332.730 ms (6.25% GC)
  maximum time:     382.852 ms (3.76% GC)
  --------------
  samples:          16
  evals/sample:     1

In [24]:
@benchmark multAB4(C, $A, $B)

BenchmarkTools.Trial: 
  memory estimate:  107.42 MiB
  allocs estimate:  6020101
  --------------
  minimum time:     204.387 ms (5.49% GC)
  median time:      315.686 ms (6.22% GC)
  mean time:        305.364 ms (6.95% GC)
  maximum time:     524.808 ms (3.06% GC)
  --------------
  samples:          17
  evals/sample:     1

In [25]:
@benchmark multAB5(C, $A, $B)

BenchmarkTools.Trial: 
  memory estimate:  108.04 MiB
  allocs estimate:  6040401
  --------------
  minimum time:     174.908 ms (5.98% GC)
  median time:      181.333 ms (6.14% GC)
  mean time:        182.261 ms (7.23% GC)
  maximum time:     206.894 ms (7.98% GC)
  --------------
  samples:          28
  evals/sample:     1

In [26]:
@benchmark multAB6(C, $A, $B)

BenchmarkTools.Trial: 
  memory estimate:  107.43 MiB
  allocs estimate:  6020401
  --------------
  minimum time:     180.923 ms (5.69% GC)
  median time:      194.118 ms (5.51% GC)
  mean time:        193.406 ms (6.75% GC)
  maximum time:     211.403 ms (11.62% GC)
  --------------
  samples:          26
  evals/sample:     1

Vemos que o tempo de execução para cada uma das ordens é muito semelhante, assim como a memória e as alocações utilizadas para realizar a operação.

## Sistemas Lineares

In [78]:
A = rand(3, 3)
b = A * ones(3)

3-element Array{Float64,1}:
 1.443654962294736
 1.9619873398663328
 2.3366920390946655

In [79]:
# O símbolo \ resolve sistemas lineares
# A \ b => Ax = b

A \ b

3-element Array{Float64,1}:
 0.9999999999999998
 1.0000000000000004
 0.9999999999999997

Sistema de exemplo:

$$ 3x_1+x_2+2x_3 = 6 $$
$$ -x_1+2x_2+x_3 = 2 $$
$$ x_1+x_2+4x_3 = 6 $$

In [80]:
A = [3.0 1 2; -1 2 1; 1 1 4]

3×3 Array{Float64,2}:
  3.0  1.0  2.0
 -1.0  2.0  1.0
  1.0  1.0  4.0

In [81]:
b = A * ones(3)

3-element Array{Float64,1}:
 6.0
 2.0
 6.0

In [82]:
m21 = A[2, 1] / A[1, 1]
m31 = A[3, 1] / A[1, 1]

# L₂ = A[2, :], L₃ = A[3, :]
A[2, 2:end] = A[2, 2:end] - m21 * A[1, 2:end]
A[2, 1] = 0.0
A[3, 2:end] = A[3, 2:end] - m31 * A[1, 2:end]
A[3, 1] = 0.0

0.0

In [83]:
b[2] = b[2] - m21 * b[1]
b[3] = b[3] - m31 * b[1]

4.0

In [84]:
A

3×3 Array{Float64,2}:
 3.0  1.0       2.0
 0.0  2.33333   1.66667
 0.0  0.666667  3.33333

In [85]:
b

3-element Array{Float64,1}:
 6.0
 4.0
 4.0

In [86]:
A \ b

3-element Array{Float64,1}:
 1.0
 1.0000000000000002
 0.9999999999999999

In [87]:
m32 = A[3, 2] / A[2, 2]
A[3, 3:end] -= m32 * A[2, 3:end]
A[3, 2] = 0.0
b[3] -= m32 * b[2]

A

3×3 Array{Float64,2}:
 3.0  1.0      2.0
 0.0  2.33333  1.66667
 0.0  0.0      2.85714

In [88]:
b

3-element Array{Float64,1}:
 6.0
 4.0
 2.8571428571428568

In [89]:
A \ b

3-element Array{Float64,1}:
 1.0
 1.0000000000000002
 0.9999999999999999

In [111]:
"""
    elim_gauss!(A, b)

Modifica A e b, fazendo a eliminição Gaussiana.
"""
function elim_gauss!(A, b)
    m, n = size(A)
    for j = 1:n
        cols = j+1:n
        for i = j+1:m
            mij = A[i, j] / A[j, j]
            A[i, cols] -= mij * A[j, cols]
            A[i, j] = 0.0
            b[i] -= mij * b[j]
        end
    end
end

elim_gauss!

In [122]:
A = rand(5, 5)
#A[:, 3] = A[:, 2] + 2 * A[:, 5]
#A[3, :] = A[2, :] + 2 * A[5, :]

b = A * ones(5)

5-element Array{Float64,1}:
 2.508337925815863
 3.45912418740253
 2.596761362611476
 2.805078163100073
 1.2515199771959236

In [123]:
A

5×5 Array{Float64,2}:
 0.267282  0.943105   0.282297    0.133495   0.882159
 0.605524  0.969112   0.776569    0.992787   0.115133
 0.283128  0.660092   0.532521    0.625996   0.495025
 0.789228  0.666978   0.563379    0.450264   0.335229
 0.732806  0.0787155  0.00785031  0.0187582  0.41339

In [124]:
elim_gauss!(A, b)

In [125]:
A

5×5 Array{Float64,2}:
 0.267282   0.943105  0.282297   0.133495   0.882159
 0.0       -1.16748   0.137029   0.690354  -1.88339
 0.0        0.0       0.193708   0.284175   0.107322
 0.0        0.0       0.0       -0.435198   1.43428
 0.0        0.0       0.0        0.0        1.72325

In [126]:
 b

5-element Array{Float64,1}:
  2.508337925815863
 -2.2234899104312165
  0.5852046231948549
  0.9990825311015119
  1.723247772696765